<a href="https://colab.research.google.com/github/edgi-govdata-archiving/EEW_TRI/blob/main/TRI_Headquarters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Who pollutes?

This notebook draws on publicly available EPA data to classify polluters (at least those who reported to the Toxic Releases Inventory in 2022) in terms of parent companies.

## Install helper code

In [ ]:
!pip install git+https://github.com/edgi-govdata-archiving/ECHO_modules &>/dev/null; # ECHO_modules
from ECHO_modules.get_data import get_echo_data # Import the get_echo_data function, which is the function that does the work of retrieving data from the SBU database
import geopandas # Geopandas is for managing spatial data (county boundaries and industrial facility points)
import folium # Folium is for mapping
import pandas

## Get TRI facilities from SBU copy of ECHO_EXPORTER


In [ ]:
sql = 'select * from "ECHO_EXPORTER" where "TRI_REPORTER" = \'Y\''
tri = get_echo_data(sql)
tri

/usr/local/lib/python3.10/dist-packages/ECHO_modules/get_data.py:33: DtypeWarning: Columns (5,119) have mixed types. Specify dtype option on import or set low_memory=False.
  ds = pd.read_csv(data_location,encoding='iso-8859-1')


,REGISTRY_ID,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,FAC_COUNTY,FAC_FIPS_CODE,FAC_EPA_REGION,FAC_INDIAN_CNTRY_FLG,...,FAC_DATE_LAST_INSPECTION_STATE,FAC_DATE_LAST_FORMAL_ACT_EPA,FAC_DATE_LAST_FORMAL_ACT_ST,FAC_DATE_LAST_INFORMAL_ACT_EPA,FAC_DATE_LAST_INFORMAL_ACT_ST,FAC_FEDERAL_AGENCY,TRI_REPORTER,FAC_IMP_WATER_FLG,EJSCREEN_FLAG_US,wkb_geometry
0,110070932452,LINDE-SWEENY,8985 WEST CR 359,SWEENY,TX,77480,BRAZORIA,48039.0,6.0,N,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,0101000020AD10000038F6ECB94CEF57C0ABECBB22F80F...
1,110070261645,PLANT 3,12270 BOD DR,ALEDO,TX,76008,TARRANT COUNTY,48439.0,6.0,N,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,N,0101000020AD100000A796ADF5456258C0C780ECF5EE5B...
2,110071139866,REVLIS CORPORATION,1075 JENKINS BLVD,AKRON,OH,44306,SUMMIT,39153.0,5.0,N,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,Y,0101000020AD1000000551F701485F54C08D28ED0DBE84...
3,110071285784,HEXAGON-AGILITY FUEL SYSTEMS,1010 CORPORATE CENTER DR.,SALISBURY,NC,28146,ROWAN COUNTY,37159.0,4.0,N,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,N,0101000020AD100000A7EA1ED95C1F54C0508BC1C3B4CF...
4,110070251611,DALLAS PLANT,2935 COMMERCE ST,DALLAS,TX,75226,DALLAS COUNTY,48113.0,6.0,N,...,NaN,NaN,09/15/2022,NaN,NaN,NaN,Y,NaN,N,0101000020AD100000A5660FB4023258C056293DD34B64...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21664,110000598997,NUCOR VULCRAFT GROUP - GRAPELAND DIV,175 COUNTY RD 2345,GRAPELAND,TX,75844,HOUSTON,48225.0,6.0,N,...,08/02/2023,06/20/2001,02/23/2001,NaN,12/16/2016,NaN,Y,NaN,N,0101000020AD10000054AC1A84B9DE57C014967840D980...
21665,110000321535,"ALERIS ROLLED PRODUCTS, INC",838 NORTH DELSEA DR,CLAYTON,NJ,8312,GLOUCESTER,34015.0,2.0,N,...,08/09/2023,04/20/2005,08/07/2023,NaN,11/12/2019,NaN,Y,Y,N,0101000020AD100000FF976BD102C652C02D5F97E13FD5...
21666,110000321580,NATIONAL REFRIGERANTS INC DEERFIELD FACILITY,661 KENYON AVE.,ROSENHAYN,NJ,8352,CUMBERLAND,34011.0,2.0,N,...,07/27/2018,NaN,08/01/2013,NaN,NaN,NaN,Y,NaN,N,0101000020AD100000FDBB3E73D6C752C06E8786C5A8BD...
21667,110000475209,BACHEM INC,3132 KASHIWA ST.,TORRANCE,CA,90505,LOS ANGELES,6037.0,9.0,N,...,12/15/2010,06/27/2016,NaN,07/19/2005,NaN,NaN,Y,NaN,Y,0101000020AD100000EBE2361AC0955DC03F52448655E8...


In [ ]:
# How many of these are majors? Only 5,191
tri.loc[tri["FAC_MAJOR_FLAG"]=="Y"]

,REGISTRY_ID,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,FAC_COUNTY,FAC_FIPS_CODE,FAC_EPA_REGION,FAC_INDIAN_CNTRY_FLG,...,FAC_DATE_LAST_INSPECTION_STATE,FAC_DATE_LAST_FORMAL_ACT_EPA,FAC_DATE_LAST_FORMAL_ACT_ST,FAC_DATE_LAST_INFORMAL_ACT_EPA,FAC_DATE_LAST_INFORMAL_ACT_ST,FAC_FEDERAL_AGENCY,TRI_REPORTER,FAC_IMP_WATER_FLG,EJSCREEN_FLAG_US,wkb_geometry
27,110000483487,PHILLIPS 66 RODEO REFINERY,1380 SAN PABLO AVENUE,RODEO,CA,94572,CONTRA COSTA COUNTY,6013.0,9.0,N,...,08/15/2023,12/05/2005,08/19/2020,NaN,08/18/2023,NaN,Y,NaN,Y,0101000020AD10000039D6C56D34905EC0840D4FAF9405...
52,110070112580,ID CASTINGS BUILDING DEMOLITION,1600 S 8TH ST,NOBLESVILLE,IN,46060,HAMILTON,18057.0,5.0,N,...,02/16/2022,NaN,04/23/2014,NaN,04/10/2007,NaN,Y,NaN,Y,0101000020AD100000EC12D55B038155C0554D10751F04...
56,110000490102,CHEMCO INCORPORATED,4191 GRANDVIEW RD,FERNDALE,WA,98248,WHATCOM,53073.0,10.0,N,...,09/30/2022,09/15/2011,11/18/2008,NaN,06/26/2018,NaN,Y,Y,N,0101000020AD100000FE7C5BB054AD5EC0C3482F6AF771...
65,110041995151,USMC CAMP LEJEUNE,1 POST LN,CAMP LEJEUNE,NC,28542,ONSLOW,37133.0,4.0,N,...,03/09/2023,09/30/2019,05/02/2017,NaN,02/22/2023,Defense: Marine Corps,Y,NaN,N,0101000020AD1000008E226B0DA55453C041F0F8F6AE55...
78,110002346239,TEXAS INSTRUMENTS,6412 US-75,SHERMAN,TX,75090,GRAYSON,48181.0,6.0,N,...,05/11/2005,07/14/2004,NaN,NaN,NaN,NaN,Y,NaN,N,0101000020AD10000004392861A62658C001BEDBBC71CA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21654,110009717912,"MASTERBRAND CABINETS, INC. - KINSTON PLANT #9",651 COLLIER LOFTIN ROAD,KINSTON,NC,28504,LENOIR,37107.0,4.0,N,...,06/29/2023,NaN,01/10/2000,NaN,11/24/2020,NaN,Y,NaN,N,0101000020AD1000001327F73B146553C00B24287E8C9D...
21658,110000310191,GE AVIATION,1000 WESTERN AVE,LYNN,MA,1905,ESSEX COUNTY,25009.0,1.0,N,...,09/28/2023,05/31/1996,02/10/1999,08/08/2017,01/06/2020,NaN,Y,Y,Y,0101000020AD100000F12F82C64CBE51C0278925E5EE39...
21660,110000462231,GOODYEAR TIRE & RUBBER HOUSTON CHEMICAL PLANT,2000 GOODYEAR DR,HOUSTON,TX,77017,HARRIS,48201.0,6.0,N,...,04/20/2022,09/07/2022,05/15/2023,03/08/2022,09/18/2018,NaN,Y,Y,Y,0101000020AD100000CEDE196D55D057C0C498F4F752B4...
21664,110000598997,NUCOR VULCRAFT GROUP - GRAPELAND DIV,175 COUNTY RD 2345,GRAPELAND,TX,75844,HOUSTON,48225.0,6.0,N,...,08/02/2023,06/20/2001,02/23/2001,NaN,12/16/2016,NaN,Y,NaN,N,0101000020AD10000054AC1A84B9DE57C014967840D980...


In [ ]:
# How many majors are there generally? 25,098
sql = 'select count(*) from "ECHO_EXPORTER" where "FAC_MAJOR_FLAG" = \'Y\''
major = get_echo_data(sql)
major

,count
0,25098


## Get TRI parent company data from EPA

In [ ]:
import requests, zipfile, io, json

url = 'https://www3.epa.gov/tri/current/US_2022.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("/content")

tri_hqs = pandas.read_csv("US_4_2022.txt", delimiter = "\t", encoding='latin1', on_bad_lines='skip') # Some problems loading one row - skipping for now
tri_hqs

<ipython-input-15-241e7ca4cc7d>:5: DtypeWarning: Columns (70,71,72,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  tri_hqs = pandas.read_csv("US_4_2022.txt", delimiter = "\t", encoding='latin1', on_bad_lines='skip') # Some problems loading one row - skipping for now


,1. REPORTING YEAR,2. TITLE OF CERTIFYING OFFICIAL,3. NAME OF CERTIFYING OFFICIAL,4. TRIFD,5. NAME,6. STREET,7. CITY,8. COUNTY,9. STATE,10. ZIP CODE,...,82. NPDES NR G,83. NPDES NR H,84. NPDES NR I,85. NPDES NR J,86. LAST YEAR THE FACILITY REPORTED,87. FIRST YEAR THE FACILITY REPORTED,88. NUMBER OF FORMS SUBMITTED,89. TOTAL NUMBER OF FORMS SUBMITTED,20231015,Unnamed: 90
0,2022,SAFETY COORDINATOR,JOHN BLOUGH,9227WCLPRT722VI,1000PALMS VISTA CHINO PLANT,72200 VISTA CHINO AVE,THOUSAND PALMS,RIVERSIDE,CA,92276,...,NaN,NaN,NaN,NaN,2022,2009,3,17,NaN,NaN
1,2022,HR/EHS ASSISTANT,TARA VILLANUEVA,4680WNLSNG345EN,105861 - NELSON GLOBAL PRODUCTS INC,3405 ENGLE RD,FORT WAYNE,ALLEN,IN,46809,...,NaN,NaN,NaN,NaN,2022,2022,1,1,NaN,NaN
2,2022,NaN,ANGELA MELE,7410WTHSTR13521,11TH STREET BATCH PLANT,13521 E 11TH ST,TULSA,TULSA,OK,74108,...,NaN,NaN,NaN,NaN,2022,2011,2,16,NaN,NaN
3,2022,REGIONAL REGULATORY MANAGER,ROBIN JOHNSTON,7516WSPCLT233HY,1I BRANCH - SPECIALTY SERVICES COMPLEX,233 HOYT ROAD,WAXAHACHIE,ELLIS,TX,75167,...,NaN,NaN,NaN,NaN,2022,2012,2,40,NaN,NaN
4,2022,PURCHASING AGENT,LOUIS SZKLANECKI,60101STYDC450SL,1ST AYD CORP,1325 GATEWAY DRIVE,ELGIN,KANE,IL,60124,...,NaN,NaN,NaN,NaN,2022,1992,1,33,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21746,2022,DIRECTOR - EHS,THOMAS FROST,7513WZRNND4894I,ZURN INDUSTRIES LLC DBA STAINLESSDRAINS.COM,4894 INTERSTATE 30 W,CADDO MILLS,HUNT,TX,75135,...,NaN,NaN,NaN,NaN,2022,2014,2,18,NaN,NaN
21747,2022,DIRECTOR - EHS,THOMAS FROST,93446ZRNND1747C,ZURN INDUSTRIES WILKINS DIV,1747 COMMERCE WAY,PASO ROBLES,SAN LUIS OBISPO,CA,93446,...,NaN,NaN,NaN,NaN,2022,1991,1,61,NaN,NaN
21748,2022,HSE MANAGER,JAIME RUIZ,77562HPRTN409EA,ZXP TECHNOLOGIES LLC,409 E WALLISVILLE RD,HIGHLANDS,HARRIS,TX,77562,...,NaN,NaN,NaN,NaN,2022,1987,1,126,NaN,NaN
21749,2022,OPS PROJECT MANAGER,AMELIA WETZLER,9480WZYGCR39LAK,ZYGO CORP EXTREME PRECISION OPTICS,3900 LAKESIDE DRIVE,RICHMOND,CONTRA COSTA,CA,94806,...,NaN,NaN,NaN,NaN,2022,2021,1,2,NaN,NaN


## Join TRI HQ data to ECHO_EXPORTER table

In [ ]:
tri.set_index("REGISTRY_ID", inplace=True)
tri_hqs.set_index("16. EPA REGISTRY ID", inplace=True)
joined = tri.join(tri_hqs)
joined

,FAC_NAME,FAC_STREET,FAC_CITY,FAC_STATE,FAC_ZIP,FAC_COUNTY,FAC_FIPS_CODE,FAC_EPA_REGION,FAC_INDIAN_CNTRY_FLG,FAC_FEDERAL_FLG,...,82. NPDES NR G,83. NPDES NR H,84. NPDES NR I,85. NPDES NR J,86. LAST YEAR THE FACILITY REPORTED,87. FIRST YEAR THE FACILITY REPORTED,88. NUMBER OF FORMS SUBMITTED,89. TOTAL NUMBER OF FORMS SUBMITTED,20231015,Unnamed: 90
1.100003e+11,"ADM ANIMAL NUTRITION, A DIVISION OF ARCHER DAN...",PR-2 KM 83,HATILLO,PR,659,HATILLO MUNICIPIO,72065.0,2.0,N,NaN,...,NaN,NaN,NaN,NaN,2022.0,1989.0,2.0,81.0,NaN,NaN
1.100003e+11,"COMPANIA CERVECERA DE PUERTO RICO, INC.",100 BLVD. ALFONSO VALDEZ,MAYAGUEZ,PR,680,MAYAGUEZ,72097.0,2.0,N,NaN,...,NaN,NaN,NaN,NaN,2022.0,1987.0,2.0,59.0,NaN,NaN
1.100003e+11,HENKEL PUERTO RICO INC,9 V QUILINCHINI AVE,SABANA GRANDE,PR,637,SABANA GRANDE,72121.0,2.0,N,NaN,...,NaN,NaN,NaN,NaN,2022.0,1987.0,2.0,148.0,NaN,NaN
1.100003e+11,PREPA AGUIRRE,ROAD #3 KM 152.3,AGUIRRE,PR,00704,SALINAS,72123.0,2.0,N,NaN,...,NaN,NaN,NaN,NaN,2022.0,1998.0,8.0,207.0,NaN,NaN
1.100003e+11,MADERAS TRATADAS (FORMER BORICUA WOOD),CARR. 865 KM. 5.5,TOA BAJA,PR,949,TOA BAJA,72137.0,2.0,N,NaN,...,NaN,NaN,NaN,NaN,2022.0,1987.0,2.0,85.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.100714e+11,PURE ENCAPSULATIONS LLC,490 BOSTON POST RD,SUDBURY,MA,1776,MIDDLESEX COUNTY,25017.0,1.0,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.100714e+11,"OPW FUELING COMPONENTS, LLC",9393 PRINCETON GLENDALE RD,HAMILTON,OH,45011,BUTLER COUNTY,39017.0,5.0,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.100714e+11,"TIMBER AUTOMATION, LLC",400 AVIATION PLZ,HOT SPRINGS NATIONAL PARK,AR,71913,GARLAND COUNTY,5051.0,6.0,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.100714e+11,TRINITY STERILE,201 KILEY DR,SALISBURY,MD,21801,WICOMICO COUNTY,24045.0,3.0,N,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Top 20 companies with the most TRI facilities in 2022

In [ ]:
joined.groupby(by=["27. SUBMITTED STANDARDIZED PARENT COMPANY NAME"])[["FAC_NAME"]].count().sort_values(by="FAC_NAME", ascending=False).head(20)

,FAC_NAME
27. SUBMITTED STANDARDIZED PARENT COMPANY NAME,
CRH AMERICAS INC,314
CEMEX INC,170
BERKSHIRE HATHAWAY INC,159
US DEPARTMENT OF DEFENSE,122
ARGOS USA CORP,96
TYSON FOODS INC,85
KOCH INDUSTRIES INC,83
HOLCIM PARTICIPATIONS (US) INC,83
TITAN AMERICA LLC,71


## Example: Where are KOCH INDUSTRIES INC facilities?

In [ ]:
from ECHO_modules.utilities import mapper

mapper(joined.loc[joined["27. SUBMITTED STANDARDIZED PARENT COMPANY NAME"] == "KOCH INDUSTRIES INC"])

/usr/local/lib/python3.10/dist-packages/ECHO_modules/utilities.py:457: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  location = [df.mean()["FAC_LAT"], df.mean()["FAC_LONG"]]
/usr/local/lib/python3.10/dist-packages/ECHO_modules/utilities.py:457: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  location = [df.mean()["FAC_LAT"], df.mean()["FAC_LONG"]]


## Which companies had the worst violation records over the past 12 quarters?

In [ ]:
# For now, we will do a quick calculation using ECHO_EXPORTER: CWA Quarters in Non-Compliance / # of Facilities
# In the future, we could do a more thorough search of enforcement records

violations = joined.loc[joined["NPDES_FLAG"] == "Y"] # First, filter to CWA-regulated facilities
average = violations.groupby(by=["27. SUBMITTED STANDARDIZED PARENT COMPANY NAME"])[["CWA_QTRS_WITH_NC"]].agg(["mean", "count"])
filtered = average.loc[average["CWA_QTRS_WITH_NC"]["count"] > 10] # Limit results to companies with > 10 facilities
filtered.sort_values(by=("CWA_QTRS_WITH_NC","mean"), ascending=False).head(20)

CWA_QTRS_WITH_NC      
                                                           mean count
27. SUBMITTED STANDARDIZED PARENT COMPANY NAME                       
DUKE ENERGY CORP                                       5.631579    19
ARDAGH HOLDINGS USA INC                                5.071429    14
THOMAS CONCRETE INC                                    5.058824    17
DARLING INGREDIENTS INC                                5.000000    14
CLEVELAND-CLIFFS INC                                   5.000000    18
GREIF INC                                              4.583333    12
FORD MOTOR CO                                          4.066667    15
WESTROCK CO                                            3.789474    19
SOUTHERN CO                                            3.750000    12
AMERICAN BATH GROUP LLC                                3.692308    13
JBS USA FOOD CO                                        3.685714    35
DUPONT DE NEMOURS INC                                  3.583333    12
LAND O LAKES INC                                       3.531250    32
UFP INDUSTRIES INC                                     3.526316    19
PERDUE FARMS INC                                       3.318182    22
INTERNATIONAL PAPER CO                                 3.277778    18
PHILLIPS 66 CO                                         3.225806    31
US DEPARTMENT OF ENERGY                                3.166667    12
DOW INC                                                3.153846    13
ARGOS USA CORP                                         3.098039    51